In [16]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' # uncomment this line if omp error occurs on OSX for python 3
os.environ['OMP_NUM_THREADS']='12' # set number of OpenMP threads to run in parallel
os.environ['MKL_NUM_THREADS']='12' # set number of MKL threads to run in parallel

In [17]:
import time
from quspin.operators import hamiltonian, quantum_operator, exp_op  # Hamiltonians and operators
from quspin.basis import spin_basis_1d  # Hilbert space spin basis
import numpy as np  # generic math functions
from quspin.tools.evolution import ExpmMultiplyParallel

参数表

In [18]:
time_start = time.time()
n_qubit = 4
a = 1
mass = 1
g = 1
mu = 0
nH = 4 # number of ham

basis = spin_basis_1d(n_qubit,pauli=-1)

pauli_x = np.array([[0,1],[1,0]]) # Pauli matrix
pauli_y = np.array([[0,-1j],[1j,0]])
pauli_z = np.array([[1,0],[0,-1]])

定义相互作用形式

In [19]:
# define one Pauli matrices
def h_n(Interaction_form: str, position: int, Coefficient=1):
    """this function is one Pauli matrices, return hamiltonian.
    need choose basis first!!!
    first parameter is interaction form, you can input 'x','y','z','I'.
    second parameter is coefficient before,
    last two parameter is lattice position,
     """
    h_list = [[Coefficient, position]]
    h_op = [[Interaction_form, h_list]]
    dynamic = []
    ham = hamiltonian(static_list=h_op, dynamic_list=dynamic,
                      dtype=np.complex128, basis=basis, check_herm=False, check_symm=False)
    return ham

# define two Pauli matrices interaction
def h_nn(Interaction_form: str, position1: int, position2: int, Coefficient=1):
    """this function is two Pauli matrices interaction, return hamiltonian.
    need choose basis first!!!
    first parameter is interaction form, you can input 'xx','yy','zz'...
    second parameter is coefficient before,
    last two parameter is lattice position,
     """
    h_list = [[Coefficient, position1, position2]]
    h_op = [[Interaction_form, h_list]]
    dynamic = []
    ham = hamiltonian(static_list=h_op, dynamic_list=dynamic,
                      dtype=np.complex128, basis=basis, check_herm=False, check_symm=False)
    return ham

# Jordan-Wigner transf.
def JWtransf(i:int):
    """J-W transf. phi_i = 1/2 * (X_i - i*Y_i) * sum_k=0^i-1 (-i*Z_k)"""
    res = (h_n('x',i) - h_n('y',i,1j))/2
    for k in range(i): # sum部分
        res = res * h_n('z',k,1j)
    return res

# define phi dagger phi
def pdaggerp(LN_i:int,LN_j:int):
    """直接手算了psidagger psi得到的具体形式，args LN_i(LN_j) mean LN_i(LN_j) th lattice number: LN_i*m + j"""
    if LN_i==LN_j:
        return (h_n('I',LN_i) + h_n('z',LN_i)) / 2
    elif LN_i>LN_j:
        temp = -1j/4 * (h_n('x',LN_i) + h_n('y',LN_i,1j)) * (h_n('x',LN_j) - h_n('y',LN_j,1j))
        for k in range(LN_j+1,LN_i):
            temp *= h_n('z',k,1j)
        return temp
    elif LN_i<LN_j:
        temp = 1j/4 * (h_n('x',LN_i) + h_n('y',LN_i,1j)) * (h_n('x',LN_j) - h_n('y',LN_j,1j))
        for k in range(LN_i+1,LN_j):
            temp *= h_n('z',k,-1j)
        return temp

def pdaggerp_m_hc(LN_i:int,LN_j:int): # J-W transf. 这个算符是反厄密的
    return pdaggerp(LN_i,LN_j) - pdaggerp(LN_j,LN_i)

def pdaggerp_p_hc(LN_i:int,LN_j:int): # J-W transf. 这个算符是厄密的
    return pdaggerp(LN_i,LN_j) + pdaggerp(LN_j,LN_i)

构造哈密顿量

In [20]:
# subsystem Hamiltonian
basis = spin_basis_1d(n_qubit,pauli=-1)
Hk = 0
for n in range(n_qubit-1):
    Hk += pdaggerp_m_hc(n,n+1)
Hk += pdaggerp_m_hc(n_qubit-1,0)
Hk *= -1j/(2*a)

Hm = 0
for n in range(n_qubit):
    Hm += (-1)**n * pdaggerp(n,n)
Hm *= mass

HI = 0
for n in range(int(n_qubit/2)):
    temp=0
    for i in range(2):
        temp += (-1)**i * pdaggerp(2*n+i,2*n+i)
    HI += temp * temp
HI *= -g/a

Hmu = 0
for n in range(n_qubit):
    Hmu += pdaggerp(n,n)
Hmu *= -mu

ham = Hk + Hm + HI + Hmu

In [29]:
delta_t = 0.1
U = exp_op(O=ham,a=-1j*delta_t).get_mat()
UT = U.getH()

res = U.dot(UT)
for i in range(2**n_qubit):
    for j in range(2**n_qubit):
        if abs(res[i,j])>1e-10:
            print("Error at position:",i,j,"value:",res[i,j])

Error at position: 0 0 value: (1+0j)
Error at position: 1 1 value: (1.0000000000000002+0j)
Error at position: 2 2 value: (1.0000000000000004+0j)
Error at position: 3 3 value: (0.9999999999999999+0j)
Error at position: 4 4 value: (1.0000000000000002+0j)
Error at position: 5 5 value: (0.9999999999999994+0j)
Error at position: 6 6 value: (0.9999999999999999+0j)
Error at position: 7 7 value: (1.0000000000000002+0j)
Error at position: 8 8 value: (1.0000000000000004+0j)
Error at position: 9 9 value: (1.0000000000000002+0j)
Error at position: 10 10 value: (1.0000000000000004+0j)
Error at position: 11 11 value: (1.0000000000000004+0j)
Error at position: 12 12 value: (0.9999999999999997+0j)
Error at position: 13 13 value: (1.0000000000000002+0j)
Error at position: 14 14 value: (1.0000000000000004+0j)
Error at position: 15 15 value: (1+0j)
